In [1]:
!pip install boto3
!pip install faker
!pip install requests
!pip install opensearch-py

In [2]:
import json
from faker import Faker
import boto3
import random
import time
import requests

In [4]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

# Get credentials to sign the requests
region = "ap-northeast-2"
# credentials = boto3.Session().get_credentials()
# auth = AWSV4SignerAuth(credentials, region)

aos_host = "search-refine-test-7upkcissxfxv3qc2wewcrvreke.aos.ap-northeast-2.on.aws"

auth = ("master", "Passw0rd!")

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

# Test the OpenSearch client
aos_client.cluster.get_settings(flat_settings=True)

{'persistent': {'cluster.ignore_dot_indexes': 'true',
  'cluster.routing.allocation.awareness.force.zone.values': 'xx-xxxxx-xx',
  'cluster.routing.allocation.disk.watermark.flood_stage': '9.824504852294922gb',
  'cluster.routing.allocation.disk.watermark.high': '21.613910675048828gb',
  'cluster.routing.allocation.disk.watermark.low': '24.561262130737305gb',
  'cluster.routing.allocation.load_awareness.provisioned_capacity': '1',
  'cluster.routing.allocation.load_awareness.skew_factor': '50.0',
  'cluster_manager.throttling.thresholds.auto-create.value': '200',
  'cluster_manager.throttling.thresholds.cluster-reroute-api.value': '50',
  'cluster_manager.throttling.thresholds.cluster-update-settings.value': '50',
  'cluster_manager.throttling.thresholds.create-component-template.value': '50',
  'cluster_manager.throttling.thresholds.create-data-stream.value': '50',
  'cluster_manager.throttling.thresholds.create-index-template-v2.value': '50',
  'cluster_manager.throttling.thresholds.

In [7]:
def generate_and_send_vpc_flowlogs(
    client=aos_client,
    aos_endpoint="https://search-refine-test-7upkcissxfxv3qc2wewcrvreke.aos.ap-northeast-2.on.aws",
    num_records=1000,
):
    """
    Generates fake VPC flow logs and sends them to a Kinesis Firehose stream.

    Args:
        firehose_stream_name (str): The name of the Kinesis Firehose stream.
        num_records (int): The number of flow log records to generate.
    """
    fake = Faker()

    # Generate 3 unique VPC IDs
    vpc_ids = set()
    while len(vpc_ids) < 5:
        vpc_ids.add(f"vpc-{fake.uuid4()[:8]}")

    # Generate 2048 unique source IP addresses
    src_ips = set()
    while len(src_ips) < 100:
        src_ips.add(fake.ipv4())

    # Generate 128 unique destination IP addresses
    dst_ips = set()
    while len(dst_ips) < 10:
        dst_ips.add(fake.ipv4())

    for _ in range(num_records):
        # Randomly select a VPC ID from the 3 generated
        vpc_id = random.choice(list(vpc_ids))

        # Randomly select a source IP address from the 2048 generated
        src_addr = random.choice(list(src_ips))

        # Randomly select a destination IP address from the 128 generated
        dst_addr = random.choice(list(dst_ips))

        # Generate fake VPC flow log data
        version = 2
        account_id = fake.uuid4()[:12]
        subnet_id = f"subnet-{fake.uuid4()[:8]}"
        instance_id = f"i-{fake.uuid4()[:10]}"
        interface_id = f"eni-{fake.uuid4()[:10]}"
        src_port = random.randint(49152, 65535)
        dst_port = random.randint(1, 65535)
        protocol = random.choice([6, 17])
        packets = random.randint(1, 100)
        bytes_ = random.randint(100, 10000)
        start = int(time.time()) - random.randint(60, 3600)
        end = start + random.randint(1, 60)
        action = random.choice(["ACCEPT", "REJECT"])
        log_status = "OK"

        # Create the flow log record in JSON format
        record = {
            "version": version,
            "account_id": account_id,
            "vpc_id": vpc_id,
            "subnet_id": subnet_id,
            "instance_id": instance_id,
            "interface_id": interface_id,
            "src_addr": src_addr,
            "dst_addr": dst_addr,
            "src_port": src_port,
            "dst_port": dst_port,
            "protocol": protocol,
            "packets": packets,
            "bytes": bytes_,
            "start": start,
            "end": end,
            "action": action,
            "log_status": log_status,
        }

        # print(record)
        client.index(index="vpcflowlogs", body=record)

        # print(f"Sent record to Kinesis Firehose: {response['RecordId']}")

    print(f"Sent {num_records} records to Amazon OpenSearch")

In [8]:
generate_and_send_vpc_flowlogs(num_records=5)

{'version': 2, 'account_id': '1dcd5440-16e', 'vpc_id': 'vpc-cc9751a9', 'subnet_id': 'subnet-4711dccc', 'instance_id': 'i-db75dc97-6', 'interface_id': 'eni-c5a5a3ef-d', 'src_addr': '136.9.208.254', 'dst_addr': '167.94.20.205', 'src_port': 61720, 'dst_port': 39476, 'protocol': 17, 'packets': 54, 'bytes': 7663, 'start': 1715069284, 'end': 1715069287, 'action': 'ACCEPT', 'log_status': 'OK'}
{'version': 2, 'account_id': '736ce491-07a', 'vpc_id': 'vpc-2efbf833', 'subnet_id': 'subnet-19e57b2d', 'instance_id': 'i-fd84df91-9', 'interface_id': 'eni-ad6fc302-3', 'src_addr': '188.186.88.49', 'dst_addr': '116.55.188.85', 'src_port': 53173, 'dst_port': 61876, 'protocol': 6, 'packets': 17, 'bytes': 7001, 'start': 1715070459, 'end': 1715070483, 'action': 'REJECT', 'log_status': 'OK'}
{'version': 2, 'account_id': 'd1230269-c53', 'vpc_id': 'vpc-6bf1bde7', 'subnet_id': 'subnet-3da718c1', 'instance_id': 'i-cac03285-d', 'interface_id': 'eni-9c23e573-3', 'src_addr': '141.157.248.153', 'dst_addr': '55.107.19